

## ** Aplicación de Filtros a Telegrafía (CW)**

Este informe consolida el análisis de diseño, rendimiento y viabilidad de los filtros FIR e IIR para el filtrado digital de la señal de telegrafía (CW). El análisis se centra en aislar el tono de audiofrecuencia (AF) de $939.91  Hz$ en un receptor cuya portadora es de $7.025  MHz$.

La información se presenta en el formato original solicitado, sin supresión ni resumen.

---

### **1\. Contexto y Especificaciones de Filtrado**

La tarea crítica de los filtros es la **selectividad final** en la etapa de audio, operando sobre la señal demodulada.

| Parámetro | Valor | Detalle |
| :---- | :---- | :---- |
| **Frecuencia de Portadora (RF)** | $7.025  MHz$ | Frecuencia de la señal de radiofrecuencia (RF). |
| **Frecuencia del Tono (AF)** | $939.91  Hz$ | Frecuencia central de la banda de paso ($F\_C$). |
| **Frecuencia de Muestreo ($F\_S$)** | $44100  Hz$ | Frecuencia de operación para el procesamiento digital. |
| **Ancho de Banda Óptimo** | $50  Hz$ | Ancho de banda que logró la **mayor disminución de ruido**. |
| **Anchos de Banda Analizados (BW)** | $50, 100, 250, 500  Hz$ | Rangos de análisis. |
| **Ancho de Transición ($\\Delta F$)** | $0.7  Hz$ | Característica de diseño constante. |

---

### **2\. Análisis Detallado de Polos y Ceros ($P/Z$) para Todos los BWs**

El diagrama de Polos y Ceros en el plano Z define la estabilidad y la respuesta en frecuencia de cada filtro.

#### **A. Filtros FIR (Kaiser y Parks-McClellan/Remez)**

Para todos los anchos de banda ($50  Hz$ a $500  Hz$), el **Orden es fijo** en $N=3000$.

| Elemento | Ubicación | Análisis y Consecuencia (Todos los BWs) |
| :---- | :---- | :---- |
| **Polos ($P$)** | $3000$ polos fijos en el $z=0$ (origen). | **Estabilidad y Fase Lineal:** La colocación de todos los polos en el origen garantiza la **estabilidad inherente** del filtro (sistema de respuesta finita) y la **Fase Lineal** (retardo de grupo constante de $\\tau\_g = 1500.5$ muestras). |
| **Ceros ($Z$)** | $3000$ ceros simétricos en el plano Z. | **Selectividad:** La selectividad se obtiene por la ubicación de los ceros **sobre o muy cerca del círculo unidad** en las frecuencias de la banda de detención ($w\_s$), forzando la atenuación. A medida que el **BW aumenta** (e.g., $50  Hz \\to 500  Hz$), la región de ceros en el círculo unidad se extiende para cubrir la banda de paso ampliada, manteniendo la estructura de fase lineal gracias a la simetría. |

#### **B. Filtros IIR (Chebyshev I y Cauer/Elíptico)**

El orden ($N$) varía para cada BW, buscando el menor orden para la especificación ($\\alpha\_p=0.5  dB$).

| BW (Hz) | Chebyshev I (N) | Cauer/Elíptico (N) |
| :---- | :---- | :---- |
| **50** | $N=7$ | **$N=5$** |
| **100** | $N=9$ | $N=6$ |
| **250** | $N=14$ | $N=8$ |
| **500** | $N=20$ | $N=10$ |

| Elemento | Ubicación General | Análisis y Consecuencia (Todos los BWs) |
| :---- | :---- | :---- |
| **Polos ($P$)** | **Dentro del círculo unidad, cerca del borde.** | **Eficiencia y Fase No Lineal:** Los polos se colocan cerca de la unidad en la vecindad de $939.91  Hz$ para obtener una respuesta fuerte con bajo orden. La proximidad de los polos al borde es la causa directa del **Retardo de Grupo variable** (fase no lineal). A medida que el **BW aumenta**, el orden $N$ también aumenta, requiriendo más polos para definir la banda de paso ampliada. |
| **Ceros ($Z$) (IIR Cauer)** | **Sobre el círculo unidad** en las frecuencias de la banda de detención ($w\_s$). | **Atenuación Óptima:** El Cauer utiliza la colocación precisa de ceros sobre el círculo unidad para generar *nulos* (atenuación extrema) en las bandas de detención. Esto le permite lograr la especificación $\\alpha\_s$ con el **mínimo orden** ($N=5$ para $50  Hz$ BW). |

---

### **3\. Tabla de Rendimiento Numérico (BW=50 Hz)**

El rendimiento en el BW óptimo de $50  Hz$ es la base de la comparación:

| Filtro | Orden (N) | Rizo Máximo en BP (dB) | Atenuación Mínima en BS (dB) | Retardo de Grupo Promedio (muestras) |
| :---- | :---- | :---- | :---- | :---- |
| **FIR Kaiser** | $N=1000$ (Fijo) | $0.9116$ | $159.6$ | $499.5$ |
| **FIR Parks-McClellan** | $N=1000$ (Fijo) | $0.1448$ | $103.6$ | $499.5$ |
| **IIR Chebyshev I** | $N=7$ | $0.5000$ (Fijo) | $139.3$ | $-2.7$ |
| **IIR Cauer/Elíptico** | $N=5$ | $0.5000$ (Fijo) | $146.9$ | $-8.1$ |

---

### **4\. Análisis de Viabilidad en ESP-32 (Implementación)**

La **carga computacional** debe permitir el procesamiento en tiempo real a $44100  Hz$ en el ESP-32. El tiempo de procesamiento por muestra no debe exceder $22.6  μs$ ($1/44100  Hz$).

#### **Filtros IIR (Cauer/Elíptico y Chebyshev I)**

| Parámetro | IIR Cauer/Elíptico (50 Hz BW) | Carga Computacional |
| :---- | :---- | :---- |
| **Orden ($N$)** | $5$ | $Mínimo$ |
| **Operaciones/Muestra** | $≈ 30$ operaciones | **Carga Mínima.** El procesamiento es casi instantáneo y fácilmente manejado por el ESP-32. |
| **Viabilidad en ESP32** | **Excelente.** Garantiza el procesamiento en tiempo real con mínima latencia. |  |

#### **Filtros FIR (Kaiser y Parks-McClellan)**

| Parámetro | FIR (50 Hz BW) | Carga Computacional |
| :---- | :---- | :---- |
| **Orden ($N$)** | $3000$ | $Prohibitivo$ |
| **Operaciones/Muestra** | $≈ 6000$ operaciones (por Convolución) | **Carga Máxima.** El tiempo de procesamiento ($≈ 6000$ multiplicaciones/sumas) excede el intervalo de muestreo ($≈ 22.6  μs$). |
| **Viabilidad en ESP32** | **Nula.** La aplicación sufriría **latencia acumulada y pérdida de datos** (underflow) al no poder mantener la tasa de $44100  Hz$. |  |

---

### **5\. Conclusión Final: Determinación del Mejor Filtro Práctico **

La elección se basa en la **eficiencia, la estabilidad práctica y la viabilidad** para la plataforma ESP-32, considerando que el **$50  Hz$ BW** es el óptimo para la reducción de ruido.

1. **FIR Descartado:** El orden $N=3000$ hace que el filtro sea inviable para la implementación en ESP-32.  
2. **IIR Elegido:** La decisión se centra en el más eficiente de los IIR.

#### **El Filtro Recomendado es el IIR Cauer/Elíptico (BW \= 50 Hz)**

| Aspecto | Razón |
| :---- | :---- |
| **Viabilidad en ESP32** | Su orden $N=5$ es el más bajo de todos, garantizando el uso mínimo de CPU y la ejecución en tiempo real. |
| **Selectividad Máxima** | El análisis de $P/Z$ confirma que la ubicación de sus ceros sobre el círculo unidad le permite lograr $146.9  dB$ de atenuación, superando incluso al FIR Remez ($103.6  dB$). |
| **Fidelidad Suficiente** | La fase no lineal del IIR es el *trade-off* necesario. Sin embargo, para un tono simple de CW, esta distorsión es **inaudible** y no afecta la detección de la señal. |

**El IIR Cauer/Elíptico** es la solución más robusta y eficiente para el filtrado de CW en sistemas embebidos.